# The Complete Effects of Coors Field

## Outline

- Check Coors Effect
    - Altitude at Coors vs. other stadiums
    - Offense numbers at Coors vs. other stadiums
        - AVG, ISO, K%, BB%, RA
        - Coors vs. All Other Parks (AOP)
    - Statcast offense numbers at Coors vs. other stadiums (is EV different? is distance to EV different?)
        - xwOBA / wOBA at every park
        - Avg hit distance by LA/EV for AOP
        - Gap in hit distance by LA/EV for Coors vs. AOP
        - Distribution in LA for Coors vs. AOP
        - Distribution in EV for Coors vs. AOP
    - 1993-2001 (pre-humidor) vs. 2002-2024 (post-humidor)
    - Park factors from FanGraphs
- Check Coors Hangover Effect
- Additional effects
    - Fielding errors
    - Strike % for pitchers
    - Pitch movement
    - Effect over time
        - Across season
        - Over multiple seasons
        - Check players changing teams
        - Temperature impact
- Redo WAR estimates for notable Rockies
- Check which pitchers/batters should perform better and do perform better at Coors


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import requests
#import time
from matplotlib import colors
import seaborn as sns
from pybaseball import team_batting, statcast
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.width = 0
pd.options.display.max_colwidth = 200
import warnings
warnings.filterwarnings(action='ignore')

## Coors Effect

Altitude for every park

In [2]:
df = statcast('2024-01-01', '2024-12-31')

This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 76/76 [00:37<00:00,  2.04it/s]


In [28]:
df['events'].unique()

array(['strikeout', nan, 'field_out', 'walk', 'single', 'double',
       'sac_fly', 'catcher_interf', 'force_out', 'hit_by_pitch',
       'fielders_choice', 'field_error', 'home_run',
       'grounded_into_double_play', 'double_play',
       'strikeout_double_play', 'fielders_choice_out', 'truncated_pa',
       'sac_bunt', 'triple', 'triple_play', 'sac_fly_double_play'],
      dtype=object)

In [ ]:
len(df[df['events'].isin(['strikeout', 'field_out', 'sac_fly', 'force_out', 'fielders_choice_out'])])
df[df['events'].isin(['grounded_into_double_play', 'double_play', 'strikeout_double_play', 'sac_bunt'])]

21272

In [62]:
df[(df['events'].notna()) & (df['events'] == 'home_run')].head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
337,FF,2024-10-30,95.5,-1.78,5.85,"Brasier, Ryan",519317,518489,home_run,hit_into_play,<NA>,<NA>,<NA>,<NA>,3,Giancarlo Stanton homers (7) on a fly ball to right field.,W,R,R,NYY,LAD,X,<NA>,fly_ball,0,0,2024,-0.58,1.28,0.38,3.2,<NA>,<NA>,<NA>,0,3,Bot,209.84,67.33,<NA>,<NA>,<NA>,<NA>,7.03848,-138.875004,-4.241431,-9.165377,33.440477,-14.857633,3.75,1.79,385,105.4,27,96.0,2124,6.8,775296,669257,518692,666158,571970,669242,606192,571771,605141,53.72,0.886,1.757,2.0,1,0,3,6,24,1,4-Seam Fastball,4,0,4,0,0,5,5,0,Strategic,Standard,215,0.037,1.114,81.6,8.1,3.389,-1.114,105.4,4,4,0.902,0.902,36,34,37,35,1,1,2,1,<NA>,<NA>,1.24,0.58,0.58,39.0,3.604828,10.124572,21.205651,45.309768,17.749705
181,FF,2024-10-30,93.3,-2.26,5.61,"Flaherty, Jack",665862,656427,home_run,hit_into_play,<NA>,<NA>,<NA>,<NA>,8,Jazz Chisholm Jr. homers (2) on a fly ball to right center field.,W,L,R,NYY,LAD,X,<NA>,fly_ball,2,1,2024,-0.38,1.38,0.09,2.22,<NA>,<NA>,<NA>,1,1,Bot,207.47,62.49,<NA>,<NA>,<NA>,<NA>,6.92289,-135.666115,-6.025083,-6.19178,29.569625,-14.002229,3.66,1.75,397,103.8,37,94.2,2230,6.8,775296,669257,518692,666158,571970,669242,606192,571771,605141,53.68,0.514,1.056,2.0,1,0,3,6,7,4,4-Seam Fastball,2,0,2,0,0,3,3,0,Standard,Standard,209,0.078,1.536,<NA>,<NA>,2.034,-1.536,103.8,2,2,0.724,0.724,28,26,29,26,1,0,5,1,<NA>,<NA>,1.24,0.38,-0.38,30.7,<NA>,<NA>,<NA>,<NA>,<NA>
217,FF,2024-10-30,93.7,-2.23,5.59,"Flaherty, Jack",592450,656427,home_run,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,Aaron Judge homers (3) on a fly ball to right center field. Juan Soto scores.,W,R,R,NYY,LAD,X,<NA>,fly_ball,0,0,2024,-0.44,1.42,-0.03,2.9,<NA>,<NA>,665742,1,1,Bot,193.54,50.49,<NA>,<NA>,<NA>,<NA>,6.719067,-136.260474,-4.350148,-7.047926,31.483467,-13.77022,4.01,1.84,403,108.9,28,94.2,2228,6.7,775296,669257,518692,666158,571970,669242,606192,571771,605141,53.78,0.986,2.019,2.0,1,0,3,6,6,1,4-Seam Fastball,0,0,0,0,0,2,2,0,Standard,Standard,210,0.174,1.436,75.9,7.6,3.931,-1.436,108.9,0,0,0.55,0.55,28,32,29,32,1,0,5,1,<NA>,<NA>,1.19,0.44,0.44,29.5,9.486577,2.538882,36.480168,40.677454,24.522149
105,FF,2024-10-29,94.2,-1.19,6.13,"Honeywell, Brent",650402,641703,home_run,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,Gleyber Torres homers (2) on a fly ball to right center field. Austin Wells scores. Alex Verdugo scores.,W,R,R,NYY,LAD,X,<NA>,fly_ball,1,1,2024,-0.41,1.28,-0.21,2.17,669224,<NA>,657077,1,8,Bot,191.6,66.31,<NA>,<NA>,<NA>,<NA>,3.43008,-137.048688,-7

In [32]:
col_df = df[(df.home_team == 'COL') & (df['events'].notna()) & (df['events'] != 'truncated_pa') & (df['estimated_woba_using_speedangle'] != '<NA>')]
in_play_df = df[(df['home_team'] != 'COL') & (df['events'].notna()) & (df['events'] != 'truncated_pa') & (df['estimated_woba_using_speedangle'] != '<NA>')]

In [80]:
metrics = ['BABIP', 'HR%', 'ISO', 'K%', 'BB%', 'R/PA']
col_metrics = ["{:.3}".format(col_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(col_df[col_df['events'] == 'home_run']) / len(col_df)),
               "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['iso_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(col_df[col_df['events'] == 'strikeout']) / len(col_df)),
               "{:.1%}".format(len(col_df[col_df['events'] == 'walk']) / len(col_df)),
               "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['post_bat_score'].mean() - col_df[col_df['description'] == 'hit_into_play']['bat_score'].mean()).lstrip('0')]
mlb_metrics = ["{:.3}".format(in_play_df[(in_play_df['description'] == 'hit_into_play') & (in_play_df['events'] != 'home_run')]['babip_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(in_play_df[in_play_df['events'] == 'home_run']) / len(in_play_df)),
               "{:.3}".format(in_play_df[in_play_df['description'] == 'hit_into_play']['iso_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(in_play_df[in_play_df['events'] == 'strikeout']) / len(in_play_df)),
               "{:.1%}".format(len(in_play_df[in_play_df['events'] == 'walk']) / len(in_play_df)),
               "{:.3}".format(in_play_df[in_play_df['description'] == 'hit_into_play']['post_bat_score'].mean() - in_play_df[in_play_df['description'] == 'hit_into_play']['bat_score'].mean()).lstrip('0')]

#babip = [col_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean(),
#         in_play_df[(in_play_df['description'] == 'hit_into_play') & (in_play_df['events'] != 'home_run')]['babip_value'].mean()]
#hr_perc = [len(col_df[col_df['events'] == 'home_run']) / len(col_df), 
#           len(in_play_df[in_play_df['events'] == 'home_run']) / len(in_play_df)]
#iso = [col_df[col_df['description'] == 'hit_into_play']['iso_value'].mean(),
#       in_play_df[in_play_df['description'] == 'hit_into_play']['iso_value'].mean()]
#k_perc = [len(col_df[col_df['events'] == 'strikeout']) / len(col_df), 
#          len(in_play_df[in_play_df['events'] == 'strikeout']) / len(in_play_df)]
#bb_perc = [len(col_df[col_df['events'] == 'walk']) / len(col_df), 
#           len(in_play_df[in_play_df['events'] == 'walk']) / len(in_play_df)]
#r_per_pa = [col_df[col_df['description'] == 'hit_into_play']['post_bat_score'].mean() - col_df[col_df['description'] == 'hit_into_play']['bat_score'].mean(),
#            in_play_df[in_play_df['description'] == 'hit_into_play']['post_bat_score'].mean() - in_play_df[in_play_df['description'] == 'hit_into_play']['bat_score'].mean()]

off_table = pd.DataFrame()
off_table[''] = metrics
off_table['Coors'] = col_metrics
off_table['All Other Parks'] = mlb_metrics
#off_table['BABIP'] = babip
#off_table['HR%'] = hr_perc
#off_table['ISO'] = iso
#off_table['K%'] = k_perc
#off_table['BB%'] = bb_perc
#off_table['R/PA'] = r_per_pa

print(off_table.to_string(index=False))

#off_table.style.format({'BABIP': '{:.3}', 
#                        'HR%': '{:.1%}',
#                        'ISO': '{:.3}',
#                        'K%': '{:.1%}',
#                        'BB%': '{:.1%}',
#                        'R/PA': '{:.3}'})
        

## Strikeouts
#print("K%")
#print("Coors:", "{:.2%}".format(len(col_df[col_df['events'] == 'strikeout']) / len(col_df)))
#print("All other parks:", "{:.2%}".format(len(in_play_df[in_play_df['events'] == 'strikeout']) / len(in_play_df)))
#
## Walks
#print("BB%")
#print("Coors:", "{:.2%}".format(len(col_df[col_df['events'] == 'walk']) / len(col_df)))
#print("All other parks:", "{:.2%}".format(len(in_play_df[in_play_df['events'] == 'walk']) / len(in_play_df)))
#
## BABIP
#print("BABIP")
#print("Coors:", "{:.3}".format(col_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean()))
#print("All other parks:", "{:.3}".format(in_play_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean()))
#
## ISO
#print("ISO")
#print("Coors:", "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['iso_value'].mean()))
#print("All other parks:", "{:.3}".format(in_play_df[in_play_df['description'] == 'hit_into_play']['iso_value'].mean()))
#
## RA
#print("R/PA")
#print("Coors:", "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['post_bat_score'].mean() - col_df[col_df['description'] == 'hit_into_play']['bat_score'].mean()))
#print("All other parks:", "{:.3}".format(in_play_df[in_play_df['description'] == 'hit_into_play']['post_bat_score'].mean() - in_play_df[in_play_df['description'] == 'hit_into_play']['bat_score'].mean()))

      Coors All Other Parks
BABIP  .325            .289
  HR%  3.8%            2.9%
  ISO  .249            .195
   K% 22.7%           23.7%
  BB%  9.0%            8.1%
 R/PA  .196            .161


In [83]:
home_metrics = ["{:.3}".format(col_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(col_df[col_df['events'] == 'home_run']) / len(col_df)),
               "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['iso_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(col_df[col_df['events'] == 'strikeout']) / len(col_df)),
               "{:.1%}".format(len(col_df[col_df['events'] == 'walk']) / len(col_df)),
               "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['post_bat_score'].mean() - col_df[col_df['description'] == 'hit_into_play']['bat_score'].mean()).lstrip('0')]
away_metrics = ["{:.3}".format(in_play_df[(in_play_df['away_team'] == 'COL') & (in_play_df['description'] == 'hit_into_play') & (in_play_df['events'] != 'home_run')]['babip_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(in_play_df[(in_play_df['away_team'] == 'COL') & (in_play_df['events'] == 'home_run')]) / len(in_play_df[(in_play_df['away_team'] == 'COL')])),
               "{:.3}".format(in_play_df[(in_play_df['away_team'] == 'COL') & (in_play_df['description'] == 'hit_into_play')]['iso_value'].mean()).lstrip('0'),
               "{:.1%}".format(len(in_play_df[(in_play_df['away_team'] == 'COL') & (in_play_df['events'] == 'strikeout')]) / len(in_play_df[(in_play_df['away_team'] == 'COL')])),
               "{:.1%}".format(len(in_play_df[(in_play_df['away_team'] == 'COL') & (in_play_df['events'] == 'walk')]) / len(in_play_df[(in_play_df['away_team'] == 'COL')])),
               "{:.3}".format(in_play_df[(in_play_df['away_team'] == 'COL') & (in_play_df['description'] == 'hit_into_play')]['post_bat_score'].mean() - in_play_df[(in_play_df['away_team'] == 'COL') & (in_play_df['description'] == 'hit_into_play')]['bat_score'].mean()).lstrip('0')]

#babip = [col_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean(),
#         in_play_df[(in_play_df['description'] == 'hit_into_play') & (in_play_df['events'] != 'home_run')]['babip_value'].mean()]
#hr_perc = [len(col_df[col_df['events'] == 'home_run']) / len(col_df), 
#           len(in_play_df[in_play_df['events'] == 'home_run']) / len(in_play_df)]
#iso = [col_df[col_df['description'] == 'hit_into_play']['iso_value'].mean(),
#       in_play_df[in_play_df['description'] == 'hit_into_play']['iso_value'].mean()]
#k_perc = [len(col_df[col_df['events'] == 'strikeout']) / len(col_df), 
#          len(in_play_df[in_play_df['events'] == 'strikeout']) / len(in_play_df)]
#bb_perc = [len(col_df[col_df['events'] == 'walk']) / len(col_df), 
#           len(in_play_df[in_play_df['events'] == 'walk']) / len(in_play_df)]
#r_per_pa = [col_df[col_df['description'] == 'hit_into_play']['post_bat_score'].mean() - col_df[col_df['description'] == 'hit_into_play']['bat_score'].mean(),
#            in_play_df[in_play_df['description'] == 'hit_into_play']['post_bat_score'].mean() - in_play_df[in_play_df['description'] == 'hit_into_play']['bat_score'].mean()]

off_table = pd.DataFrame()
off_table[''] = metrics
off_table['Home'] = home_metrics
off_table['Away'] = away_metrics
#off_table['BABIP'] = babip
#off_table['HR%'] = hr_perc
#off_table['ISO'] = iso
#off_table['K%'] = k_perc
#off_table['BB%'] = bb_perc
#off_table['R/PA'] = r_per_pa

print(off_table.to_string(index=False))

#off_table.style.format({'BABIP': '{:.3}', 
#                        'HR%': '{:.1%}',
#                        'ISO': '{:.3}',
#                        'K%': '{:.1%}',
#                        'BB%': '{:.1%}',
#                        'R/PA': '{:.3}'})
        

## Strikeouts
#print("K%")
#print("Coors:", "{:.2%}".format(len(col_df[col_df['events'] == 'strikeout']) / len(col_df)))
#print("All other parks:", "{:.2%}".format(len(in_play_df[in_play_df['events'] == 'strikeout']) / len(in_play_df)))
#
## Walks
#print("BB%")
#print("Coors:", "{:.2%}".format(len(col_df[col_df['events'] == 'walk']) / len(col_df)))
#print("All other parks:", "{:.2%}".format(len(in_play_df[in_play_df['events'] == 'walk']) / len(in_play_df)))
#
## BABIP
#print("BABIP")
#print("Coors:", "{:.3}".format(col_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean()))
#print("All other parks:", "{:.3}".format(in_play_df[(col_df['description'] == 'hit_into_play') & (col_df['events'] != 'home_run')]['babip_value'].mean()))
#
## ISO
#print("ISO")
#print("Coors:", "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['iso_value'].mean()))
#print("All other parks:", "{:.3}".format(in_play_df[in_play_df['description'] == 'hit_into_play']['iso_value'].mean()))
#
## RA
#print("R/PA")
#print("Coors:", "{:.3}".format(col_df[col_df['description'] == 'hit_into_play']['post_bat_score'].mean() - col_df[col_df['description'] == 'hit_into_play']['bat_score'].mean()))
#print("All other parks:", "{:.3}".format(in_play_df[in_play_df['description'] == 'hit_into_play']['post_bat_score'].mean() - in_play_df[in_play_df['description'] == 'hit_into_play']['bat_score'].mean()))

       Home  Away
BABIP  .325  .275
  HR%  3.8%  3.4%
  ISO  .249  .229
   K% 22.7% 27.4%
  BB%  9.0%  8.2%
 R/PA  .196  .165
